In [0]:

%skip
%sql
create catalog if not exists project

In [0]:
%skip
%sql
create schema if not exists project.bronze

In [0]:
%skip
%sql
CREATE TABLE IF NOT EXISTS project.bronze.Orders

In [0]:
display(dbutils.fs.ls("abfss://files@stazure2025.dfs.core.windows.net/bronze/Orders/"))

In [0]:
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://files@stazure2025.dfs.core.windows.net/bronze/Orders/")
df.display()


In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .option("mergeSchema", "true")\
  .option("delta.enableChangeDataFeed", "true") \
  .saveAsTable("project.bronze.Orders")

In [0]:
%sql
select * from project.bronze.Orders


In [0]:
df_sliver=spark.sql(f"select * from project.bronze.Orders")
df_sliver.display()

In [0]:
from pyspark.sql.functions import *

In [0]:
df_sliver.dropDuplicates()

In [0]:
df_sliver.columns

In [0]:
a=df_sliver.filter([sum(col(c).isNull().cast("int")).alias(c) for c in df_sliver.columns])


In [0]:
null_values=df_sliver.select([
    sum(col(c).isNull().cast("int")).alias(c) 
    for c in df_sliver.columns
    ])
null_values.show()

In [0]:
df_sliver.count()

In [0]:
null_counts_df = df_sliver.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df_sliver.columns
])

null_counts_df.first().asDict()

In [0]:
df_sliver=df_sliver.drop('PickedByPersonID','BackorderOrderID','Comments','DeliveryInstructions','InternalComments','PickingCompletedWhen')

In [0]:
%skip
null_counts = null_counts_df.first().asDict()

cols_to_delete = [c for c, v in null_counts.items() if v == 0]

df_wrong = df_sliver.drop(*cols_to_delete)

In [0]:
df_sliver.display()

In [0]:
df_sliverss=df_sliver.select(min('OrderDate').alias("staring_date"),max('OrderDate').alias("end date")).show()

In [0]:
df=df_sliver.orderBy(col("LastEditedWhen").desc())
df.limit(10).display()


In [0]:
df_sliver.count()

In [0]:
df_sliver.write.format("delta").mode("overwrite").option("mergeSchema", "true").option("delta.enableChangeDataFeed", "true").saveAsTable("project.silver.oooo")